# Keras PortraitNet

In [ ]:
# magic notebook options
%load_ext autoreload
%autoreload 2

In [ ]:
# library imports
import os
# ------ choose GPUs to be used -------
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" #gpu to be used

from mobilenetv3 import net_modified as net, data
from utils import *

import subprocess
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import callbacks, optimizers
print("Tensorflow version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# model name
model_name = 'portraitnet'

# dictionary to config the network parameters
config = dict(
           model_name = model_name,
           classes    = 5
)

# # metric to be monitored during training (e.g. val_mask_output_loss)
monitor = 'val_mask_output_loss' 

# paths to train and validation sets
folders_train = ['path to dataset 1', 'path to dataset 2'] 
folders_test  = ['path to testset 1', 'path to testset 2'] # used during validation

In [ ]:
# use mixed precision training (not neccessary)
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

## Dataset

In [ ]:
# build data loader
loader = data.DataLoader(folders_train, folders_test)

In [ ]:
# visualize generator
loader.view_data('train')

In [ ]:
# visualize generator
loader.view_data('test')

## Train

Here all the train steps are made, such as loading the model and define the train options.

In [ ]:
# build model
with tf.distribute.MirroredStrategy().scope():
    model = net.Model(config)
model.summary()

In [ ]:
# start training
model.fit(
    loader.flow('train', batch_size=32),
    steps_per_epoch=loader.data_size('train')//32,

    epochs=50,
    callbacks=[callbacks.TerminateOnNaN(),
               callbacks.ModelCheckpoint(f'results/{model.name}.h5',
                                         monitor=monitor, mode='min',
                                         verbose=1, save_best_only=True),
               callbacks.EarlyStopping(monitor=monitor, patience=5, verbose=1,
                                       mode='min', restore_best_weights=True),
               callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=2,
                                           verbose=1, mode='min', min_lr=1e-6),
               callbacks.CSVLogger(f'results/{model.name}_history.csv')],

    validation_data=loader.flow('test', batch_size=2),
    validation_steps=loader.data_size('test')//2,

    use_multiprocessing=True
)

## Evaluation

Here we evaluate the model and write the results on a csv file that will contain all of the experiments results.

In [ ]:
# load weight's model from training
model.load_weights(f'results/{model.name}.h5')

In [ ]:
# save results on csv file
losses = model.evaluate_generator(loader.flow('test', batch_size=1), steps=loader.data_size('test'), verbose=1)

path_eval = f'results/{model.name}_validation.csv'
with open(path_eval, 'a') as file:
    if not os.path.exists(path_eval): # header
        file.write(f"Model name,{','.join(model.metrics_names)}\n")
        
    file.write(f"{model.name},{','.join([str(l) for l in losses])}\n")

## Testing

Here are codes to test the model visualizing the results of input and predictions

In [ ]:
# predict on the data from train or test set
loader.predict_data(model, 'test')

In [ ]:
# predictions on a non-labeled testset
path_images = glob('path to a testset/*.jpg')

plt.figure(figsize=(40,40))
for i,p in enumerate(path_images):
    x = imread(p, resize=(224,224))
    
    p = loader.predict_input(model, x)

    plt.subplot(len(path_images)//4+1,4,i+1)
    plt.imshow(np.hstack([x,p]))
    plt.axis('off')
plt.savefig(f'results/{model.name}_testset.jpg')